In [ ]:
import os 
import pandas as pd
import requests
import xml.etree.ElementTree as ET

In [9]:


# def downloadfile(downloadpath, url):
    
#     # 發送 HTTP 請求，獲取 XML 資料
#     response = requests.get(url)
    
#     # 如果請求成功（狀態碼為 200），則寫入檔案
#     if response.status_code == 200:
#         with open(downloadpath, 'wb') as file:
#             file.write(response.content)
#         print(f"XML 檔案已成功下載到 {downloadpath}")
#     else:
#         print(f"請求失敗，狀態碼：{response.status_code}")
#     return response.text

def downloadfile(downloadpath, url):
    # 發送 HTTP 請求，獲取 XML 資料
    response = requests.get(url)
    
    # 如果請求成功（狀態碼為 200），則寫入檔案並返回內容
    if response.status_code == 200:
        with open(downloadpath, 'wb') as file:
            file.write(response.content)
        print(f"XML 檔案已成功下載到 {downloadpath}")
        
        # 返回 XML 檔案的內容（文本）
        return response.text
    else:
        print(f"請求失敗，狀態碼：{response.status_code}")
        return None



In [3]:
import xml.etree.ElementTree as ET

def parse_vd_xml(xml_content):
    """解析VD的XML並將其轉換為DataFrame"""
    # 解析 XML 內容
    root = ET.fromstring(xml_content)
    
    # 找到所有的 <VD> 節點
    vds = root.findall('.//VD')
    
    # 用來儲存每個VD的資料
    vd_data = []
    
    # 提取每個 VD 的資料
    for vd in vds:
        vd_info = {
            "VDID": vd.find('.//VDID').text,
            "SubAuthorityCode": vd.find('.//SubAuthorityCode').text,
            "BiDirectional": vd.find('.//BiDirectional').text,
            "VDType": vd.find('.//VDType').text,
            "LocationType": vd.find('.//LocationType').text,
            "DetectionType": vd.find('.//DetectionType').text,
            "PositionLon": vd.find('.//PositionLon').text,
            "PositionLat": vd.find('.//PositionLat').text,
            "RoadID": vd.find('.//RoadID').text,
            "RoadName": vd.find('.//RoadName').text,
            "RoadClass": vd.find('.//RoadClass').text,
            "LocationMile": vd.find('.//LocationMile').text,
        }
        
        # 處理 <DetectionLink> 資料
        detection_links = []
        for link in vd.findall('.//DetectionLinks//DetectionLink'):
            detection_link_info = {
                "LinkID": link.find('.//LinkID').text,
                "Bearing": link.find('.//Bearing').text,
                "RoadDirection": link.find('.//RoadDirection').text,
                "LaneNum": link.find('.//LaneNum').text,
                "ActualLaneNum": link.find('.//ActualLaneNum').text,
            }
            detection_links.append(detection_link_info)
        
        # 添加 DetectionLinks 資料
        vd_info["DetectionLinks"] = detection_links
        
        # 將此 VD 的資料加入到總資料列表
        vd_data.append(vd_info)
    
    # 將資料轉換為 DataFrame
    df = pd.json_normalize(vd_data, "DetectionLinks", ["VDID", "SubAuthorityCode", "BiDirectional", "VDType", "LocationType", "DetectionType", "PositionLon", "PositionLat", "RoadID", "RoadName", "RoadClass", "LocationMile"], errors="ignore")
    
    return df

In [4]:
urls = {"VD" : "https://tisvcloud.freeway.gov.tw/history/motc20/VD.xml",
        "ETagPair" : "https://tisvcloud.freeway.gov.tw/history/motc20/ETagPair.xml",
        "ETag" : "https://tisvcloud.freeway.gov.tw/history/motc20/ETag.xml",
        "CCTV" : "https://tisvcloud.freeway.gov.tw/history/motc20/CCTV.xml"}

In [5]:
downloadfolder = create_folder(os.path.join(os.getcwd(),'高公局靜態資料清單'))

In [ ]:
downloadpath = os.path.join(downloadfolder, "VD.xml")
xml  = downloadfile(downloadpath, urls['VD'])
df = parse_vd_xml(xml)

In [ ]:
xml

In [ ]:
df

In [13]:
def main():

    downloadfolder = create_folder(os.path.join(os.getcwd(),'高公局靜態資料清單'))
        # 迴圈下載每個檔案
    for file_name, url in urls.items():
        downloadpath = os.path.join(downloadfolder, f"{file_name}.xml")
        downloadfile(downloadpath, url)


In [ ]:
if __name__ == "__main__":
    main()

In [ ]:


# 使用方法
url = 
downloadpath = "VD.xml"  # 可以根據需要修改下載的路徑和檔案名稱
downloadfile(downloadpath, url)


In [7]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import os 

def create_folder(folder_name):
    """建立資料夾"""
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    return folder_name

def downloadfile(downloadpath, url="https://tisvcloud.freeway.gov.tw/history/motc20/VD.xml"):
    """
    下載 XML 檔案內容。
    :param downloadpath: 儲存 XML 檔案的路徑
    :param url: XML 檔案的下載 URL
    :return: XML 檔案內容
    """
    response = requests.get(url)
    if response.status_code == 200:
        with open(downloadpath, 'wb') as file:
            file.write(response.content)
        print(f"XML 檔案已下載至 {downloadpath}")
        return response.content
    else:
        raise Exception(f"無法下載檔案，HTTP 狀態碼: {response.status_code}")

def parse_vd_xml(xml_content):
    """
    解析 XML 檔案為 DataFrame。
    :param xml_content: XML 檔案內容
    :return: 轉換後的 DataFrame
    """
    namespace = {'ns': 'http://traffic.transportdata.tw/standard/traffic/schema/'}
    root = ET.fromstring(xml_content)
    
    data = []
    for vd in root.findall('.//ns:VDs/ns:VD', namespace):
        vdid = vd.find('ns:VDID', namespace).text
        sub_authority = vd.find('ns:SubAuthorityCode', namespace).text
        bi_directional = vd.find('ns:BiDirectional', namespace).text
        position_lon = vd.find('ns:PositionLon', namespace).text
        position_lat = vd.find('ns:PositionLat', namespace).text
        road_name = vd.find('ns:RoadName', namespace).text if vd.find('ns:RoadName', namespace) is not None else None
        location_mile = vd.find('ns:LocationMile', namespace).text if vd.find('ns:LocationMile', namespace) is not None else None

        # 每個 VD 的 DetectionLinks
        for link in vd.findall('.//ns:DetectionLink', namespace):
            link_id = link.find('ns:LinkID', namespace).text
            bearing = link.find('ns:Bearing', namespace).text
            road_direction = link.find('ns:RoadDirection', namespace).text
            lane_num = link.find('ns:LaneNum', namespace).text
            actual_lane_num = link.find('ns:ActualLaneNum', namespace).text
            
            data.append({
                "VDID": vdid,
                "SubAuthorityCode": sub_authority,
                "BiDirectional": bi_directional,
                "PositionLon": position_lon,
                "PositionLat": position_lat,
                "RoadName": road_name,
                "LocationMile": location_mile,
                "LinkID": link_id,
                "Bearing": bearing,
                "RoadDirection": road_direction,
                "LaneNum": lane_num,
                "ActualLaneNum": actual_lane_num
            })
    
    return pd.DataFrame(data)





In [8]:


def parse_vd_xml(xml_content):
    """
    解析 XML 檔案為 DataFrame，提取所有相關欄位。
    :param xml_content: XML 檔案內容
    :return: 轉換後的 DataFrame
    """
    namespace = {'ns': 'http://traffic.transportdata.tw/standard/traffic/schema/'}
    root = ET.fromstring(xml_content)

    data = []
    for vd in root.findall('.//ns:VDs/ns:VD', namespace):
        # 基本資料
        vdid = vd.find('ns:VDID', namespace).text
        sub_authority = vd.find('ns:SubAuthorityCode', namespace).text
        bi_directional = vd.find('ns:BiDirectional', namespace).text
        vd_type = vd.find('ns:VDType', namespace).text
        location_type = vd.find('ns:LocationType', namespace).text
        detection_type = vd.find('ns:DetectionType', namespace).text
        position_lon = vd.find('ns:PositionLon', namespace).text
        position_lat = vd.find('ns:PositionLat', namespace).text
        road_id = vd.find('ns:RoadID', namespace).text if vd.find('ns:RoadID', namespace) is not None else None
        road_name = vd.find('ns:RoadName', namespace).text if vd.find('ns:RoadName', namespace) is not None else None
        road_class = vd.find('ns:RoadClass', namespace).text if vd.find('ns:RoadClass', namespace) is not None else None
        location_mile = vd.find('ns:LocationMile', namespace).text if vd.find('ns:LocationMile', namespace) is not None else None

        # 道路區間
        road_section_start = vd.find('./ns:RoadSection/ns:Start', namespace).text if vd.find('./ns:RoadSection/ns:Start', namespace) is not None else None
        road_section_end = vd.find('./ns:RoadSection/ns:End', namespace).text if vd.find('./ns:RoadSection/ns:End', namespace) is not None else None

        # DetectionLinks 資料
        for link in vd.findall('./ns:DetectionLinks/ns:DetectionLink', namespace):
            link_id = link.find('ns:LinkID', namespace).text
            bearing = link.find('ns:Bearing', namespace).text if link.find('ns:Bearing', namespace) is not None else None
            road_direction = link.find('ns:RoadDirection', namespace).text if link.find('ns:RoadDirection', namespace) is not None else None
            lane_num = link.find('ns:LaneNum', namespace).text if link.find('ns:LaneNum', namespace) is not None else None
            actual_lane_num = link.find('ns:ActualLaneNum', namespace).text if link.find('ns:ActualLaneNum', namespace) is not None else None

            # 將資料加入清單
            data.append({
                "VDID": vdid,
                "SubAuthorityCode": sub_authority,
                "BiDirectional": bi_directional,
                "VDType": vd_type,
                "LocationType": location_type,
                "DetectionType": detection_type,
                "PositionLon": position_lon,
                "PositionLat": position_lat,
                "RoadID": road_id,
                "RoadName": road_name,
                "RoadClass": road_class,
                "LocationMile": location_mile,
                "RoadSectionStart": road_section_start,
                "RoadSectionEnd": road_section_end,
                "LinkID": link_id,
                "Bearing": bearing,
                "RoadDirection": road_direction,
                "LaneNum": lane_num,
                "ActualLaneNum": actual_lane_num,
            })

    return pd.DataFrame(data)


In [9]:
# 主程式
if __name__ == "__main__":
    downloadfolder = create_folder(os.path.join(os.getcwd(),'高公局靜態資料清單'))
    download_path = os.path.join(downloadfolder, 'VD.xml')
    csv_output_path = os.path.join(downloadfolder, 'VD.csv')
    # 下載 XML
    xml_content = downloadfile(download_path)
    # 解析 XML
    df = parse_vd_xml(xml_content)
    # 儲存為 CSV
    df.to_csv(csv_output_path, index=False, encoding="big5")
    print(f"DataFrame 已儲存為 {csv_output_path}")
    print(df.columns)

XML 檔案已下載至 /Users/zhangkaijie/Desktop/Work/THI/THI-git/THI-GIS-Tool/高公局靜態資料清單/VD.xml
DataFrame 已儲存為 /Users/zhangkaijie/Desktop/Work/THI/THI-git/THI-GIS-Tool/高公局靜態資料清單/VD.csv
Index(['VDID', 'SubAuthorityCode', 'BiDirectional', 'VDType', 'LocationType',
       'DetectionType', 'PositionLon', 'PositionLat', 'RoadID', 'RoadName',
       'RoadClass', 'LocationMile', 'RoadSectionStart', 'RoadSectionEnd',
       'LinkID', 'Bearing', 'RoadDirection', 'LaneNum', 'ActualLaneNum'],
      dtype='object')
